# 이기적 빅데이터 분석기사 실기

## 작업형 2유형 - 기출문제 6회(23.06.24)

In [1]:
# 학생들의 생활패턴과 신체, 거주지역 등에 대한 데이터
# 학생 530명에 대한 학습용 데이터를 이용하여 학생 구분(유치원생, 초등학생, 중학생, 고등학생, 대학생) 예측 모형을 만든 후, 
# 이를 평가용 데이터에 적용하여 얻은 230명 학생의 구분을 다음과 같은 형식의 CSV 파일로 만드시오
# 구분예측
# 중학생
# 초등학생
# 대학생
# 고등학생
# ...

### 데이터 확인

In [2]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score

In [3]:
# 데이터 불러오기
train = pd.read_csv('C:/Users/chunc/Desktop/빅분기 연습/dataset-main/data_6_2_train.csv')
test = pd.read_csv('C:/Users/chunc/Desktop/빅분기 연습/dataset-main/data_6_2_test.csv')
display(train.head())
display(test.head())

,일평균수면시간,일평균학습시간,일평균스마트폰사용시간,키,몸무게,체격,기초대사량,거주지역,구분
0,7.35,7.74,4.8,Tall,52.050,L,1499.4,동구,고등학생
1,7.05,8.10,1.2,Tall,51.300,M,1624.4,서구,중학생
2,7.35,7.74,3.0,Tall,51.285,L,1499.4,동구,고등학생
3,8.58,6.66,4.8,Short,25.545,XS,1249.5,서구,초등학생
4,7.05,8.10,4.8,Tall,54.150,M,1624.4,서구,대학생


,일평균수면시간,일평균학습시간,일평균스마트폰사용시간,키,몸무게,체격,기초대사량,거주지역
0,7.97,7.11,1.2,Tall,51.375,L,1749.3,북구
1,9.80,5.76,1.2,Short,28.710,XS,1749.3,북구
2,7.66,7.38,1.2,Tall,36.915,L,1624.4,남구
3,9.50,5.94,3.0,Short,24.045,XS,1624.4,동구
4,6.44,8.82,3.0,Tall,45.180,S,1499.4,서구


### EDA

In [4]:
# train데이터 타입 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 530 entries, 0 to 529
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   일평균수면시간      530 non-null    float64
 1   일평균학습시간      530 non-null    float64
 2   일평균스마트폰사용시간  530 non-null    float64
 3   키            530 non-null    object 
 4   몸무게          530 non-null    float64
 5   체격           530 non-null    object 
 6   기초대사량        530 non-null    float64
 7   거주지역         530 non-null    object 
 8   구분           530 non-null    object 
dtypes: float64(5), object(4)
memory usage: 37.4+ KB


In [5]:
# train데이터 기초통계 확인
train.describe(include = 'object')

,키,체격,거주지역,구분
count,530,530,530,530
unique,2,4,4,5
top,Tall,XS,남구,유치원생
freq,277,253,145,140


In [6]:
# train 데이터 결측치 확인
train.isnull().sum()

일평균수면시간        0
일평균학습시간        0
일평균스마트폰사용시간    0
키              0
몸무게            0
체격             0
기초대사량          0
거주지역           0
구분             0
dtype: int64

In [7]:
# train 데이터 타겟 값 분포 확인
train['구분'].value_counts()

구분
유치원생    140
초등학생    121
고등학생    120
대학생      78
중학생      71
Name: count, dtype: int64

In [8]:
# test 데이터 확인
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   일평균수면시간      230 non-null    float64
 1   일평균학습시간      230 non-null    float64
 2   일평균스마트폰사용시간  230 non-null    float64
 3   키            230 non-null    object 
 4   몸무게          230 non-null    float64
 5   체격           230 non-null    object 
 6   기초대사량        230 non-null    float64
 7   거주지역         230 non-null    object 
dtypes: float64(5), object(3)
memory usage: 14.5+ KB


In [9]:
# test 데이터 기초통계 확인
test.describe(include = 'object')

,키,체격,거주지역
count,230,230,230
unique,2,4,4
top,Short,XS,북구
freq,126,126,74


In [10]:
# test 데이터 결측치 확인
test.isnull().sum()

일평균수면시간        0
일평균학습시간        0
일평균스마트폰사용시간    0
키              0
몸무게            0
체격             0
기초대사량          0
거주지역           0
dtype: int64

### 데이터 전처리

In [11]:
# train 데이터의 타겟 값 처리
target = train.pop('구분')

In [12]:
# train/test 데이터 원핫 인코딩
print(train.shape, test.shape)
train = pd.get_dummies(train)
test = pd.get_dummies(test)
print(train.shape, test.shape)

(530, 8) (230, 8)
(530, 15) (230, 15)


### 검증용 데이터 생성

In [13]:
X_train, X_val, y_train, y_val = train_test_split(train, target, test_size = 0.2, random_state = 0)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(424, 15) (106, 15) (424,) (106,)


### 모델링

In [14]:
# 분류 모델 생성
rf = RandomForestClassifier(random_state = 0)
rf.fit(X_train, y_train)
pred = rf.predict(X_val)

f1_score(y_val, pred, average = 'macro')

0.9317983948418732

In [15]:
# 테스트 데이터로 예측, 결과 파일 생성
pred = rf.predict(test)
result = pd.DataFrame({'pred': pred})
result.to_csv('result.csv', index = False, encoding = "utf-8-sig")